- Ok so this specific order of imports cannot be changed or else you get some bizarre error  
- If an error pops up when importing try restarting the kernel

In [1]:
import numpy as np
import sys
import pandas as pd
import pickle

from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering


from preprocess import *
from utilities import *
from constants import *
from supervised_sentiment_analysis import *
from skip_thought_vectors import *
from graphing import *



ImportError: No module named 'nltk'

- Encode preprocessed answers as thought vectors, and add them as a column to merged_results
- Some (two, specifically) of the entries are encoded as nan because they are too long, these are filtered out by vectorized_results

In [ ]:
merged_results = pickle.load(open('merged_results.pickle', 'rb'))
merged_results['Valid Vector'] = merged_results['Skip Thought Vector'].apply(lambda x: ~np.isnan(x).any())
removed_results = merged_results[~merged_results['Valid Vector']]
merged_results = merged_results[merged_results['Valid Vector']]

In [ ]:
merged_results['Categorical Tag'].unique()

In [ ]:
[print(f"{len(answer.split())} \n{answer}") for answer in removed_results['Answer']]
# Model really doesn't like long responses for some reason

In [ ]:
get_nn('i wrote pseudocode', merged_results)

In [ ]:
N_CLUSTERS = 20

In [ ]:
labeled_results = merged_results[merged_results['Categorical Tag'] != 'no tag']
q1_labeled_results = labeled_results[labeled_results['Question'] == Q1]
q1_features = np.array(q1_labeled_results['Skip Thought Vector'].tolist())
q1_labels = np.array(q1_labeled_results['Categorical Tag'])
q1_labeled_results[q1_labeled_results['Categorical Tag'] == 'negative'].shape

In [ ]:
plot_model_confusion_matrix(q1_features, q1_labels, model = 'rfc')

In [ ]:
plot_model_confusion_matrix(q1_features, q1_labels, model = 'svc')

In [ ]:
TRIALS = 1
SPLITS = 100

In [ ]:
features = np.array(q1_labeled_results['Skip Thought Vector'].tolist())
labels = np.array(q1_labeled_results['Categorical Tag'])

conf_matrices = get_conf_matrices(features, labels, clfs = ['svc', 'gnb', 'rfc'], trials = TRIALS, splits = SPLITS, test_size = 0.3)

In [ ]:
def get_average_heatmaps(confusion_matrices, normalize = True, trials = 1, splits = 10, figsize = (8,8)):
    average_confusion_matrices = copy.deepcopy(confusion_matrices)
    clf_names = {
        'svc': 'Support Vector Classifier',
        'gnb': 'Gaussian Naive Bayes',
        'rfc': 'Random Forest Classifier'
    }
    l = len(confusion_matrices)
    fig, axs = plt.subplots(l, squeeze=False, figsize = (figsize[0],figsize[1] * l), gridspec_kw={'hspace': 0.2, 'wspace': 0})
    pos = 0
    
    for clf, matrices in average_confusion_matrices.items():
        average_confusion_matrix = sum(matrices)/len(matrices)
        print(average_confusion_matrix)
        stdev_matrix = np.std(matrices, axis=0)
        figs = 2
        def round_int(string):
            return string[:figs] if '.' not in string[:figs] else string[:figs+1]
        vfunc = np.vectorize(round_int)
        def signif(x, p):
            x = np.asarray(x)
            x_positive = np.where(np.isfinite(x) & (x != 0), np.abs(x), 10**(p-1))
            mags = 10 ** (p - 1 - np.floor(np.log10(x_positive)))
            rounded = np.round(x * mags) / mags
            vfunc = np.vectorize(lambda x: x[:-2] if len(x) == p+2 and x[-2:] == '.0' else x)
            return vfunc(rounded.astype(str))
        
    
        str_mean_matrix = signif(average_confusion_matrix, figs).astype(str)
        str_stdev_matrix = signif(stdev_matrix, figs).astype(str)
        annot_array = np.core.defchararray.add(str_mean_matrix, '\n(')
        annot_array = np.core.defchararray.add(annot_array, str_stdev_matrix)
        annot_array = np.core.defchararray.add(annot_array, ')')
        if normalize:
            average_confusion_matrix = normalized(average_confusion_matrix, axis = 1, order = 1)
        vmin = 0 if normalize else None
        vmax = 1 if normalize else None
        ax = axs[pos,0]
        print(ax)
        pos+=1
        sns.heatmap(average_confusion_matrix, cmap=plt.cm.Blues, ax = ax, 
                    vmin = vmin, vmax = vmax, 
                    square = True, annot = annot_array, 
                    xticklabels = CLASS_NAMES, yticklabels = CLASS_NAMES, fmt = '',
                    annot_kws={"fontsize":16})
        ax.set_yticklabels(rotation = 0, size = 14, labels = CLASS_NAMES)
        ax.set_xticklabels(size = 14, labels = CLASS_NAMES)
        trial_string = '' if trials == 1 else str(trials)+" trials, "
        normalized_string = 'Normalized ' if normalize else ''
        title = f"{normalized_string}Confusion Matrix using\n{clf_names[clf]}, {trial_string}{str(splits)} splits"
        ax.set_title(title, size = 18)
        ax.set_xlabel('Predicted Label', size = 16)
        ax.set_ylabel('True Label', size = 16)

svc_dict = {'svc': conf_matrices['svc']}
get_average_heatmaps(svc_dict, trials = TRIALS, splits = SPLITS, normalize = False, figsize = (8,8))

In [ ]:
cv_count = 10
get_cross_validation_scores(features, labels, svm.SVC(), cv_count = cv_count)
get_cross_validation_scores(features, labels, RandomForestClassifier(), cv_count = cv_count)
get_cross_validation_scores(features, labels, GaussianNB(), cv_count = cv_count)

In [ ]:
numerical_tags = {
    'positive': 1,
    'neutral': 0,
    'negative': -1
}
q1_numerical_tags = q1_labeled_results['Categorical Tag'].apply(lambda x: numerical_tags[x])
scatter_plot(q1_features, labels = q1_numerical_tags)

In [ ]:
svc = svm.SVC()
rfc = RandomForestClassifier()
gnb = GaussianNB()
for clf in [svc, rfc, gnb]:
    arr = np.array(get_kappa(features, labels, clf, trials = SPLITS))
    print(arr.mean(), arr.std())

In [ ]:
for clf, conf_matrix in conf_matrices.items():
    macro_F_scores = []
    print(clf)
    for conf_matrix in conf_matrix:
        macro_F_scores.append(macro_F(conf_matrix))
    print((np.mean(macro_F_scores), np.std(macro_F_scores)))

In [ ]:
q2_results = merged_results[merged_results['Question'] == Q2]

In [ ]:
n_clusters = 10
reduced_q2_encodings = reduce_dimensions(list(q2_results['Skip Thought Vector']))
agg_labels, model = get_agg_clusters(list(q2_results['Skip Thought Vector']), n_clusters = n_clusters)
q2_results['Agg Label'] = agg_labels

In [ ]:
def sample_clusters(results, cluster_count = n_clusters, sample_size = 5):
    for k in range(cluster_count):
        print(k, '\n', '\n')
        labels_k = results[results['Agg Label'] == k]
        [print(answer, '\n') for answer in labels_k['Answer'].sample(n=sample_size, replace = True)]

        
    

In [ ]:
scatter_plot(reduced_q2_encodings, labels = agg_labels, tsne = False, title = 'Reduced Dimension Skip-Thought Vectors', show_legend = True, cmap = plt.cm.nipy_spectral)

In [ ]:
filtered_q2_results = q2_results[q2_results['Agg Label'] < 4]
reduced_filtered_q2_encodings = reduce_dimensions(list(filtered_q2_results['Skip Thought Vector']))
filtered_agg_labels, filtered_model = get_agg_clusters(reduced_filtered_q2_encodings, n_clusters = n_clusters)
filtered_q2_results['Agg Label'] = filtered_agg_labels

In [ ]:
sample_clusters(filtered_q2_results)

In [ ]:
scatter_plot(reduced_filtered_q2_encodings, labels = filtered_agg_labels, tsne = False, show_legend = True, cmap = plt.cm.nipy_spectral)

In [ ]:
long_q1_labeled_results = labeled_results[labeled_results['Question'] == Q1]
long_q1_labeled_results = long_q1_labeled_results[long_q1_labeled_results['Answer'].str.split().str.len() > 3]
long_q1_features = np.array(long_q1_labeled_results['Skip Thought Vector'].tolist())
long_q1_labels = np.array(long_q1_labeled_results['Categorical Tag'])
plot_model_confusion_matrix(long_q1_features, long_q1_labels, model = 'svc')

In [ ]:
len(long_q1_labels)

In [ ]:
len(q1_labeled_results[q1_labeled_results['Answer'].str.split().str.len() > 5])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(q1_features, q1_labels, test_size=0.3)
long_X_train, long_X_test, long_y_train, long_y_test = train_test_split(long_q1_features, long_q1_labels, test_size=0.3)
clf = svm.SVC()


# fit model and predict labels
clf.fit(X_train, y_train)
y_pred=clf.predict(long_X_test)

# create confusion matrix
class_names = ['positive', 'neutral', 'negative']
conf_mat = confusion_matrix(long_y_test, y_pred, labels= class_names)

#graphing stuff
titles_options = [("Confusion matrix, without normalization", None),
                  ("Normalized confusion matrix", 'true')]
for title, normalize in titles_options:
    disp = plot_confusion_matrix(clf, long_X_test, long_y_test,
                                 display_labels=class_names,
                                 cmap=plt.cm.Blues,
                                 normalize=normalize)
    disp.ax_.set_title(title)

    print(title)
    print(disp.confusion_matrix)

plt.show()

In [ ]:
from sklearn.decomposition import PCA


def reduce_dimensions_50(X):
    PCA_50 = PCA(n_components=50)
    X_r = PCA_50.fit_transform(X)
    return X_r

pca_results = list(reduce_dimensions_50(list(q1_labeled_results['Skip Thought Vector'])))
q1_labeled_results['50 dim'] = pca_results

In [ ]:
plot_model_confusion_matrix(list(q1_labeled_results['50 dim']), list(q1_labeled_results['Categorical Tag']), model = 'gnb')